We’ll build a Multi-Skill Intelligent Research Agent that can:

 ✅ read and analyze text<br>
 ✅ perform web search (real-time using DuckDuckGo 🦆)<br>
 ✅ do math reasoning (like currency conversions or stats)<br>
 ✅ Use memory to keep context across steps<br>
 ✅ summarize and cross-combine results<br>

🚀 Goal

You’ll be able to ask something like:<br>

“Find the latest news about AI regulation in Europe, summarize it, and calculate how many years since the first major EU AI proposal in 2021.”

In [32]:
from langchain_community.llms import Ollama
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.memory import ConversationBufferMemory
import math, re, datetime

1. Connect to local model

In [3]:
llm = Ollama(model="mistral")  # you can replace with llama3, phi3, etc.

/tmp/ipykernel_50649/1810637363.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mistral")  # you can replace with llama3, phi3, etc.


2. Define custom tools

In [28]:
@tool
def summarize_text(text: str) -> str:
    """Summarize the given text into 3-5 lines."""
    sentences = re.split(r'(?<=[.!?]) +', text)
    summary = " ".join(sentences[:5])
    return f"Summary: {summary}"

@tool
def simple_calculator(expression: str) -> str:
    """Safely evaluates a simple math expression."""
    expr = expression.lower().replace("sqrt", "math.sqrt").replace("^", "**")
    try:
        result = eval(expr, {"__builtins__": None, "math": math})
        return str(result)
    except Exception as e:
        return f"Error: {e}"

@tool
def years_since_contextual(input_text: str) -> str:
    """Calculates years since a given year, considering context in the same string."""
    try:
        years = [int(y) for y in re.findall(r"20\d{2}", input_text)]
        if not years:
            return "No valid year found."
        target_year = years[0]
        ref_year = years[-1] if len(years) > 1 else datetime.datetime.now().year
        diff = ref_year - target_year
        if diff < 0:
            return f"The year {target_year} is after {ref_year}."
        elif diff == 0:
            return f"No years have passed between {target_year} and {ref_year}."
        else:
            return f"{diff} years have passed between {target_year} and {ref_year}."
    except Exception as e:
        return f"Error: {e}"

@tool
def reflect_answer(answer: str) -> str:
    """
    Reflect on the agent's answer and check for logical or contextual errors.
    If the answer seems inconsistent or wrong, provide a corrected version.
    """
    reasoning_prompt = f"""
    You are a logical critic.
    Review this answer carefully: "{answer}"
    Does it make contextual sense, especially regarding time, logic, or data consistency?
    If it's correct, say "Looks good."
    If not, rewrite it correctly and clearly explain the fix.
    """
    reflection = llm.invoke(reasoning_prompt)
    return reflection

3. Use community search tool

In [27]:
web_search = DuckDuckGoSearchRun()

Add all tools

In [29]:
tools = [summarize_text, simple_calculator, years_since_contextual, web_search, reflect_answer]

5. Add memory

In [33]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

/tmp/ipykernel_50649/3353824168.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


4. Initialize the Agent

In [35]:
agent = initialize_agent(
    tools,
    llm,
    agent_type=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    memory=memory
)

5. Ask a complex question

In [37]:
query = """
Find the latest 2025 news about inflation rates in the United States.
Summarize it in a few lines.
Then calculate the inflation increase if it rose from 3.2% in 2024 to 4.0% in 2025.
Reflect on whether this change is moderate, high, or concerning given global economic context.
"""

result = agent.run(query)

print("\n🧠 Final Combined Result:\n", result)



> Entering new AgentExecutor chain...
 I need to find the latest news about inflation rates in the United States as of 2025, summarize it, calculate the inflation increase from 2024 to 2025, and reflect on whether this change is moderate, high, or concerning given global economic context.

Action: duckduckgo_search
Action Input: Latest inflation news United States 2025
Observation: August 11, 2025 - Predicting inflation is tough due to factors like politics and supply-chain disruptions. The Consumer Price Index for all items increased by 0.3% in June 2025 , on a seasonally adjusted basis, as reported by the U.S. The annual inflation rate for the United States was 2.9% for the 12 months ending August, compared to 2.7% previously, according to U.S. Labor Department data published on September 11, 2025 . The next inflation update is scheduled for release on October 15 at 8:30 a.m. ET, providing information on the inflation rate for the 12 The Consumer Price Index for All Urban Consumers